---
toc: true
comments: true
layout: post
title: JS/Python/Flask Full Stack and User profile
description: Team teach & full example code for the JS/Python/Flask User Profile system.
courses: { compsci: {week: 20} }
type: hacks
---

## Full Python CRUD (Create/Read User Data)

<p>Following Python backend code allows entries in User to be manipulated with the following functionality:</p>

<p>class _CRUD:</p>

- post: Adds entries to User with provided properties
- get: Requests full set of data in User
- put: Update an entry in User with given id with passed properties
- delete: Removes the specified entry

<p>class _Security:</p>

- post: Returns data associated with a specified uid if the correct password is given

In [ ]:
import json
from flask import Blueprint, request, jsonify
from flask_restful import Api, Resource # used for REST API building
from datetime import datetime

from model.users import User

user_api = Blueprint('user_api', __name__,
                   url_prefix='/api/users')

# API docs https://flask-restful.readthedocs.io/en/latest/api.html
api = Api(user_api)

class UserAPI:        
    class _CRUD(Resource):  # User API operation for Create, Read.
        def post(self): # Creates an entry in User with the given properties and check for overlap/formatting errors
            ''' Read data for json body '''
            body = request.get_json()
            
            ''' Avoid garbage in, error checking '''
            # validate name
            name = body.get('name')
            if name is None or len(name) < 2:
                return {'message': f'Name is missing, or is less than 2 characters'}, 400
            # validate uid
            uid = body.get('uid')
            if uid is None or len(uid) < 2:
                return {'message': f'User ID is missing, or is less than 2 characters'}, 400
            # look for password
            password = body.get('password')

            active_classes = body.get('active_classes')
            if active_classes is None or len(active_classes) == 0:
                return {'message': 'Active class is missing'}, 400

            archived_classes = body.get('archived_classes')

            ''' #1: Key code block, setup USER OBJECT '''
            uo = User(name=name, 
                      uid=uid,
                      active_classes=active_classes,
                      archived_classes=archived_classes)
            
            ''' Additional garbage error checking '''
            # set password if provided
            if password is not None:
                uo.set_password(password)
            # set server request status    
            uo.server_needed = body.get('server_needed')
            
            ''' #2: Key Code block to add user to database '''
            # create user in database
            user = uo.create()
            # success returns json of user
            if user:
                return jsonify(user.read())
            # failure returns error
            return {'message': f'Processed {name}, either a format error or User ID {uid} is duplicate'}, 400

        def get(self): # Loads data from User for all users as JSON
            users = User.query.all()    # read/extract all users from database
            json_ready = [user.read() for user in users]  # prepare output in json
            return jsonify(json_ready)  # jsonify creates Flask response object, more specific to APIs than json.dumps
        
        def put(self, id):  # Updates user with passed id's properties in User
            ''' Read data for json body '''
            body = request.get_json()

            ''' Find user by ID '''
            user = User.query.get(id)
            if user is None:
                return {'message': f'User with ID {id} not found'}, 404

            ''' Update fields '''
            name = body.get('name')
            if name:
                user.name = name
                
            uid = body.get('uid')
            if uid:  
                user.uid = uid
                
            user.server_needed = body.get('server_needed')

            user.active_classes = body.get('active_classes')
            user.archived_classes = body.get('archived_classes')

            ''' Commit changes to the database '''
            user.update()
            return jsonify(user.read())
        
        def delete(self, id):  # Deletes user with given id, if it exists
            ''' Find user by ID '''
            user = User.query.get(id)
            if user is None:
                return {'message': f'User with ID {id} not found'}, 404

            ''' Delete user '''
            user.delete()
            return {'message': f'User with ID {id} has been deleted'}

    
    class _Security(Resource):

        def post(self):  #Return data for a given uid if the correct password is passed
            ''' Read data for json body '''
            body = request.get_json()
            
            ''' Get Data '''
            uid = body.get('uid')
            if uid is None or len(uid) < 2:
                return {'message': f'User ID is missing, or is less than 2 characters'}, 400
            password = body.get('password')
            
            ''' Find user '''
            user = User.query.filter_by(_uid=uid).first()
            if user is None or not user.is_password(password):
                return {'message': f"Invalid user id or password"}, 400
            
            ''' authenticated user '''
            return jsonify(user.read())

            
    # building RESTapi endpoint
    api.add_resource(_CRUD, '/', '/<int:id>')
    api.add_resource(_Security, '/authenticate')
    

In [ ]:
<form onsubmit="submitForm(event)">
	<label for="githubUsername">GitHub Username:</label>
	<input type="text" id="githubUsername" name="githubUsername" required /><br /><br />

	<label for="fullName">Full Name:</label>
	<input type="text" id="fullName" name="fullName" required /><br /><br />

	<label for="password">Password:</label>
	<input type="password" id="password" name="password" required /><br /><br />

	<label>Need Server:</label>
	<input type="radio" id="yes" name="serverNeeded" value="Yes" required />
	<label for="yes">Yes</label>
	<input type="radio" id="no" name="serverNeeded" value="No" required />
	<label for="no">No</label><br /><br />

	<label>Active Classes:</label><br />
	<input type="checkbox" id="APCSA" name="activeClasses" value="APCSA" />
	<label for="APCSA">APCSA</label><br />
	<input type="checkbox" id="APCSP" name="activeClasses" value="APCSP" />
	<label for="APCSP">APCSP</label><br />
	<input type="checkbox" id="CSSE" name="activeClasses" value="CSSE" />
	<label for="CSSE">CSSE</label><br /><br />

	<label>Archived Classes:</label><br />
	<input type="checkbox" id="archivedAPCSA" name="archivedClasses" value="APCSA" />
	<label for="archivedAPCSA">APCSA</label><br />
	<input type="checkbox" id="archivedAPCSP" name="archivedClasses" value="APCSP" />
	<label for="archivedAPCSP">APCSP</label><br />
	<input type="checkbox" id="archivedCSSE" name="archivedClasses" value="CSSE" />
	<label for="archivedCSSE">CSSE</label><br /><br />

	<input type="submit" value="Submit" />
</form>

<script>
	const apiUrl = "https://devops.nighthawkcodingsociety.com/api/users/";
	// const apiUrl = "http://localhost:8180/api/users/";
	let users = [];

	function fetchUsers() {
		fetch(apiUrl)
			.then((response) => response.json())
			.then((response) => {
				users = response;

				const table = document.getElementById("userTable");
				users.forEach((user, idx) => {
					const row = table.insertRow();

					row.setAttribute("data-id", user.id);
					["name", "uid", "active_classes", "archived_classes", "server_needed"].forEach(
						(field) => {
							const cell = row.insertCell();
							if (user[field] === "none") {
								users[idx][field] = "";
							}
							cell.innerText = users[idx][field];
						}
					);

					const editCell = row.insertCell();
					const editButton = document.createElement("button");
					editButton.innerHTML = "Edit";
					editButton.addEventListener("click", editUser);
					editCell.appendChild(editButton);

					const deleteCell = row.insertCell();
					const deleteButton = document.createElement("button");
					deleteButton.innerText = "Delete";
					deleteButton.addEventListener("click", () => deleteUser(user.id, row));
					deleteCell.appendChild(deleteButton);
				});
			});
	}

	function submitForm(event) {
		event.preventDefault();
		const formData = new FormData(event.target);
		const name = formData.get("fullName");
		const uid = formData.get("githubUsername");
		const password = formData.get("password");
		const serverNeeded = formData.get("serverNeeded") === "Yes";
		const activeClasses = formData.getAll("activeClasses").join(",");
		const archivedClasses = formData.getAll("archivedClasses").join(",");

		const payload = {
			name,
			uid,
			password,
			active_classes: activeClasses,
			archived_classes: archivedClasses,
			server_needed: serverNeeded,
		};

		fetch(apiUrl, {
			method: "POST",
			headers: {
				"Content-Type": "application/json",
			},
			body: JSON.stringify(payload),
		})
			.then((response) => {
				if (response.ok) {
					return response.json();
				} else {
					alert("server error");
					throw new Error("server");
				}
			})
			.then((data) => {
				const table = document.getElementById("userTable");
				const row = table.insertRow();
				row.setAttribute("data-id", data.id);
				[
					data.name,
					data.uid,
					data.active_classes,
					data.archived_classes,
					data.server_needed,
				].forEach((value) => {
					const cell = row.insertCell();
					cell.innerText = value;
				});

				const editCell = row.insertCell();
				const editButton = document.createElement("button");
				editButton.innerHTML = "Edit";
				editButton.addEventListener("click", editUser);
				editCell.appendChild(editButton);

				const deleteCell = row.insertCell();
				const deleteButton = document.createElement("button");
				deleteButton.innerText = "Delete";
				deleteButton.addEventListener("click", () => deleteUser(user.id, row));
				deleteCell.appendChild(deleteButton);

				users.push(data);
				alert("Created sucessfully!");
			})
			.catch((error) => console.error("Error:", error));
	}

	function editUser(event) {
		const id = event.currentTarget.parentElement.parentElement.getAttribute("data-id");
		document.getElementById("editId").value = id;

		const form = document.getElementById("editForm");
		const user = users.find((u) => u.id == id);

		form.querySelector("#editGithubUsername").value = user.uid;
		form.querySelector("#editFullName").value = user.name;

		document.getElementById("editYes").checked = user.server_needed;
		document.getElementById("editNo").checked = !user.server_needed;

		if (user.active_classes.length !== 0) {
			user.active_classes.split(",").forEach((cls) => {
				document.getElementById(`edit${cls}`).checked = true;
			});
		}

		if (user.archived_classes.length !== 0) {
			user.archived_classes.split(",").forEach((cls) => {
				document.getElementById(`editArchived${cls}`).checked = true;
			});
		}

		document.getElementById("editModalBackdrop").style.display = "block";
	}

	// Fetch users and ensure close modal interaction
	document.addEventListener("DOMContentLoaded", function () {
		fetchUsers();
		document.getElementById("closeModal").addEventListener("click", function () {
			document.getElementById("editModalBackdrop").style.display = "none";
		});
	});

	function submitEdit(event) {
		event.preventDefault();
		const formData = new FormData(event.target);
		const id = formData.get("editId");
		const name = formData.get("editFullName");
		const uid = formData.get("editGithubUsername");
		const serverNeeded = formData.get("editServerNeeded") === "Yes";

		const activeClasses = formData.getAll("editActiveClasses").join(",");
		const archivedClasses = formData.getAll("editArchivedClasses").join(",");

		const payload = {
			id,
			name,
			uid,
			active_classes: activeClasses,
			archived_classes: archivedClasses,
			server_needed: serverNeeded,
		};

		fetch(`${apiUrl}${id}`, {
			method: "PUT",
			headers: {
				"Content-Type": "application/json",
			},
			body: JSON.stringify(payload),
		}).then((response) => {
			if (response.ok) {
				// Update the corresponding row in the table
				const row = document.querySelector(`tr[data-id='${id}']`);
				row.cells[0].innerText = name;
				row.cells[1].innerText = uid;
				row.cells[2].innerText = activeClasses;
				row.cells[3].innerText = archivedClasses;
				row.cells[4].innerText = serverNeeded;

				// Show an alert indicating success
				alert("User information updated successfully.");
			}
		});

		document.getElementById("editModalBackdrop").style.display = "none";
	}

	function deleteUser(id, row) {
		const confirmation = prompt('Type "DELETE" to confirm.');
		if (confirmation === "DELETE") {
			fetch(`${apiUrl}${id}`, {
				method: "DELETE",
			})
				.then(() => {
					row.remove();
					alert("User deleted successfully");
				})
				.catch((error) => {
					console.error("Error:", error);
				});
		}
	}
</script>

<hr style="margin-top: 10px" />

<h2>Current Records</h2>
<table id="userTable">
	<tr>
		<th>Full Name</th>
		<th>GitHub Username</th>
		<th>Active Classes</th>
		<th>Archived Classes</th>
		<th>Server Needed</th>
		<th>Edit</th>
		<th>Delete</th>
	</tr>
</table>

<div id="editModalBackdrop" class="modal-backdrop">
	<div id="editModal" onsubmit="submitEdit(event)" class="modal-content">
		<button id="closeModal" class="close-modal">X</button>
		<form id="editForm">
			<input type="hidden" id="editId" name="editId" />

			<label for="editGithubUsername">GitHub Username:</label>
			<input type="text" id="editGithubUsername" name="editGithubUsername" /><br /><br />

			<label for="editFullName">Full Name:</label>
			<input type="text" id="editFullName" name="editFullName" /><br /><br />

			<label>Need Server:</label>
			<input type="radio" id="editYes" name="editServerNeeded" value="Yes" />
			<label for="editYes">Yes</label>
			<input type="radio" id="editNo" name="editServerNeeded" value="No" />
			<label for="editNo">No</label><br /><br />

			<label>Edit Active Classes:</label><br />
			<input type="checkbox" id="editAPCSA" name="editActiveClasses" value="APCSA" />
			<label for="editAPCSA">APCSA</label><br />
			<input type="checkbox" id="editAPCSP" name="editActiveClasses" value="APCSP" />
			<label for="editAPCSP">APCSP</label><br />
			<input type="checkbox" id="editCSSE" name="editActiveClasses" value="CSSE" />
			<label for="editCSSE">CSSE</label><br /><br />

			<label>Edit Archived Classes:</label><br />
			<input type="checkbox" id="editArchivedAPCSA" name="editArchivedClasses" value="APCSA" />
			<label for="editArchivedAPCSA">APCSA</label><br />
			<input type="checkbox" id="editArchivedAPCSP" name="editArchivedClasses" value="APCSP" />
			<label for="editArchivedAPCSP">APCSP</label><br />
			<input type="checkbox" id="editArchivedCSSE" name="editArchivedClasses" value="CSSE" />
			<label for="editArchivedCSSE">CSSE</label><br /><br />

			<input type="submit" value="Update" />
		</form>
	</div>
</div>

<style>
	.modal-backdrop {
		display: none;
		position: fixed;
		top: 0;
		left: 0;
		width: 100%;
		height: 100%;
		background-color: rgba(0, 0, 0, 0.7);
		z-index: 1;
	}

	.modal-content {
		position: absolute;
		top: 50%;
		left: 50%;
		transform: translate(-50%, -50%);
		background: #272726;
		padding: 40px;
		z-index: 2;
	}

	.close-modal {
		position: absolute;
		top: 10px;
		right: 10px;
		cursor: pointer;
		background: none;
		border: none;
		font-size: 24px;
		color: white;
	}

	.wrapper,
	section {
		max-width: 900px;
	}
</style>

In [ ]:
import json, jwt
from flask import Blueprint, request, jsonify, current_app, Response
from flask_restful import Api, Resource # used for REST API building
from datetime import datetime
from auth_middleware import token_required

from model.users import User

user_api = Blueprint('user_api', __name__,
                   url_prefix='/api/users')

# API docs https://flask-restful.readthedocs.io/en/latest/api.html
api = Api(user_api)

class UserAPI:        
    class _CRUD(Resource):  # User API operation for Create, Read.  THe Update, Delete methods need to be implemeented
        @token_required
        def post(self, current_user): # Create method
            ''' Read data for json body '''
            body = request.get_json()
            
            ''' Avoid garbage in, error checking '''
            # validate name
            name = body.get('name')
            if name is None or len(name) < 2:
                return {'message': f'Name is missing, or is less than 2 characters'}, 400
            # validate uid
            uid = body.get('uid')
            if uid is None or len(uid) < 2:
                return {'message': f'User ID is missing, or is less than 2 characters'}, 400
            # look for password and dob
            password = body.get('password')
            dob = body.get('dob')

            ''' #1: Key code block, setup USER OBJECT '''
            uo = User(name=name, 
                      uid=uid)
            
            ''' Additional garbage error checking '''
            # set password if provided
            if password is not None:
                uo.set_password(password)
            # convert to date type
            if dob is not None:
                try:
                    uo.dob = datetime.strptime(dob, '%Y-%m-%d').date()
                except:
                    return {'message': f'Date of birth format error {dob}, must be mm-dd-yyyy'}, 400
            
            ''' #2: Key Code block to add user to database '''
            # create user in database
            user = uo.create()
            # success returns json of user
            if user:
                return jsonify(user.read())
            # failure returns error
            return {'message': f'Processed {name}, either a format error or User ID {uid} is duplicate'}, 400

        @token_required
        def get(self, current_user): # Read Method
            users = User.query.all()    # read/extract all users from database
            json_ready = [user.read() for user in users]  # prepare output in json
            return jsonify(json_ready)  # jsonify creates Flask response object, more specific to APIs than json.dumps
    
    class _Security(Resource):
        def post(self):
            try:
                body = request.get_json()
                if not body:
                    return {
                        "message": "Please provide user details",
                        "data": None,
                        "error": "Bad request"
                    }, 400
                ''' Get Data '''
                uid = body.get('uid')
                if uid is None:
                    return {'message': f'User ID is missing'}, 400
                password = body.get('password')
                
                ''' Find user '''
                user = User.query.filter_by(_uid=uid).first()
                if user is None or not user.is_password(password):
                    return {'message': f"Invalid user id or password"}, 400
                if user:
                    try:
                        token = jwt.encode(
                            {"_uid": user._uid},
                            current_app.config["SECRET_KEY"],
                            algorithm="HS256"
                        )
                        resp = Response("Authentication for %s successful" % (user._uid))
                        resp.set_cookie("jwt", token,
                                max_age=3600,
                                secure=True,
                                httponly=True,
                                path='/',
                                samesite='None'  # This is the key part for cross-site requests

                                # domain="frontend.com"
                                )
                        return resp
                    except Exception as e:
                        return {
                            "error": "Something went wrong",
                            "message": str(e)
                        }, 500
                return {
                    "message": "Error fetching auth token!",
                    "data": None,
                    "error": "Unauthorized"
                }, 404
            except Exception as e:
                return {
                        "message": "Something went wrong!",
                        "error": str(e),
                        "data": None
                }, 500

            
    # building RESTapi endpoint
    api.add_resource(_CRUD, '/')
    api.add_resource(_Security, '/authenticate')
    